In [1]:
try:
  # We must install required packages if we are in Google Colab
  import google.colab
  %pip install roboticstoolbox-python>=1.0.2
except:
  # We are not in Google Colab
  # Apply custon style to notebook
  from IPython.core.display import HTML
  import pathlib
  styles_path = pathlib.Path(pathlib.Path().absolute(), "style", "style.css")
  styles = open(styles_path, "r").read()
  HTML(f"<style>{styles}</style>")

# The Manipulator Jacobian

In [2]:
# Import libraries
# We will do the imports required for this notebook here

# numpy provides import array and linear algebra utilities
import numpy as np

# the robotics toolbox provides robotics specific functionality
import roboticstoolbox as rtb

# spatial math provides objects for representing transformations
import spatialmath as sm
import spatialmath.base as smb

# use timeit to benchmark some methods
from timeit import default_timer as timer

# ansitable is a great package for printing tables in a terminal
from ansitable import ANSITable

## Utility Functions

In [6]:
# We can program these in Python
# These methods assume the inputs are correctly sized and numpy arrays

def vex(mat):
    '''
    Converts a 3x3 skew symmetric matric to a 3 vector
    '''
    return np.array([mat[2, 1], mat[0, 2], mat[1, 0]])

def skew(vec):
    '''
    Converts a 3 vector to a 3x3 skew symmetric matrix
    '''

    return np.array([
        [0, -vec[2], vec[1]],
        [vec[2], 0, -vec[0]],
        [-vec[1], vec[0], 0]
    ])

def vexa(mat):
    '''
    Converts a 4x4 augmented skew symmetric matric to a 6 vector
    '''

    return np.array([mat[0, 3], mat[1, 3], mat[2, 3], mat[2, 1], mat[0, 2], mat[1, 0]])

def skewa(vec):
    '''
    Converts a 6 vector to a 4x4 augmented skew symmetric matrix
    '''

    return np.array([
        [0, -vec[5], vec[4], vec[0]],
        [vec[5], 0, -vec[3], vec[1]],
        [-vec[4], vec[3], 0, vec[2]],
        [0, 0, 0, 0]
    ])

In [7]:
# Test our skew and vex methods out

x = np.array([1, 2, 3])

sk_x = skew(x)

print(f"x: \n{x}")
print(f"skew form of x: \n{sk_x}")
print(f"Perform vex operation on that, we should get the original a vector back: \n{vex(sk_x)}")

x: 
[1 2 3]
skew form of x: 
[[ 0 -3  2]
 [ 3  0 -1]
 [-2  1  0]]
Perform vex operation on that, we should get the original a vector back: 
[1 2 3]


In [8]:
y = np.array([1, 2, 3, 4, 5, 6])

ska_y = skewa(y)

print(f"y: \n{y}")
print(f"augmented skew form of y: \n{ska_y}")
print(f"Perform vexa operation on that, we should get the original y vector back: \n{vexa(ska_y)}")

y: 
[1 2 3 4 5 6]
augmented skew form of y: 
[[ 0 -6  5  1]
 [ 6  0 -4  2]
 [-5  4  0  3]
 [ 0  0  0  0]]
Perform vexa operation on that, we should get the original y vector back: 
[1 2 3 4 5 6]



We also need a function to extract the translational or rotational component from an $\bf{SE}(3)$ or augmented skew-symmetric matrix

<br>

<br>

<img src="img/transform_dark.png" alt="drawing" width="700"/>


_Visualisation of a homogeneous transformation matrix (the derivatives share the form of $\ \bf{T}$, except will have a 0 instead of a 1 located at $\bf{T}_{44}$). Where the matrix $\rho(\bf{T}) \in \mathbb{R}^{3 \times 3}$ of green boxes forms the rotation component, and the vector $\tau(\bf{T}) \in \mathbb{R}^{3}$ of blue boxes form the translation component. The rotation component can be extracted through the function $\rho(\cdot) : \mathbb{R}^{4\times 4} \mapsto  \mathbb{R}^{3\times 3}$, while the translation component can be extracted through the function $\tau(\cdot) : \mathbb{R}^{4\times 4} \mapsto  \mathbb{R}^{3}$._


In [10]:
def ρ(tf):
    '''
    The method extracts the rotational component from an SE3
    '''
    return tf[:3, :3]

def τ(tf): 
    '''
    The method extracts the translation component from an SE3
    '''
    return tf[:3, 3]

In [12]:
print(ρ(ska_y))
print(τ(ska_y))

[[ 0 -6  5]
 [ 6  0 -4]
 [-5  4  0]]
[1 2 3]


## Derivation of a Pure Rotation

In [13]:
# We can make Python functions which perform these derivatives

def dTRx(θ, flip):
    '''
    Calculates the derivative of an SE3 which is a pure rotation around
    the x-axis by amount θ
    '''
    # This is the [Rhat_x] matrix in the maths above
    Rhx = np.array([
        [0, 0, 0, 0],
        [0, 0, -1, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 0]
    ])

    # This is the T_R_x matrix in the maths above
    Trx = smb.trotx(θ)

    if flip:
        return Rhx.T @ Trx.T
    else:
        return Rhx @ Trx

def dTRy(θ, flip):
    '''
    Calculates the derivative of an SE3 which is a pure rotation around
    the y-axis by amount θ
    '''
    # This is the [Rhat_y] matrix in the maths above
    Rhy = np.array([
        [0, 0, 1, 0],
        [0, 0, 0, 0],
        [-1, 0, 0, 0],
        [0, 0, 0, 0]
    ])

    # This is the T_R_y matrix in the maths above
    Try = smb.troty(θ)

    if flip:
        return Rhy.T @ Try.T
    else:
        return Rhy @ Try

def dTRz(θ, flip):
    '''
    Calculates the derivative of an SE3 which is a pure rotation around
    the z-axis by amount θ
    '''
    # This is the [Rhat_z] matrix in the maths above
    Rhz = np.array([
        [0, -1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ])

    # This is the T_R_z matrix in the maths above
    Trz = smb.trotz(θ)

    if flip:
        return Rhz.T @ Trz.T
    else:
        return Rhz @ Trz
